# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
!pip3 install scipy==1.2
!pip3 install gdown
!gdown https://drive.google.com/uc?id=1ehyrYBQ5rbQQe6yL4XbLWe3FMvuVUGiL
!unzip -q Project_data.zip

     |████████████████████████████████| 26.6 MB 4.2 MB/s            
  Attempting uninstall: scipy
    Found existing installation: scipy 1.7.3
    Uninstalling scipy-1.7.3:
      Successfully uninstalled scipy-1.7.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.3.post1 requires numpy<1.20,>=1.16.0, but you have numpy 1.20.3 which is incompatible.
tpot 0.11.7 requires scipy>=1.3.1, but you have scipy 1.2.0 which is incompatible.
stumpy 1.10.2 requires scipy>=1.5, but you have scipy 1.2.0 which is incompatible.
statsmodels 0.13.1 requires scipy>=1.3, but you have scipy 1.2.0 which is incompatible.
sklearn-pandas 2.2.0 requires scipy>=1.5.1, but you have scipy 1.2.0 which is incompatible.
scikit-image 0.19.1 requires scipy>=1.4.1, but you have scipy 1.2.0 which is incompatible.
plotnine 0.8.0 requires scipy>=1.5.0, but you have scipy 1.2.0 which i

In [2]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os
import cv2

We set the random seed so that the results don't vary drastically.

In [3]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [4]:

train_doc = np.random.permutation(open('Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('Project_data/val.csv').readlines())
batch_size = 32 #experiment with the batch size

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [5]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [0,1,2,6,10,14,16,20,24,27,28,29]
#create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = int(len(folder_list)/batch_size)
        # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            x=len(img_idx)
            #batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_data = np.zeros((batch_size,x,100,100,3))
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                     
                    #cropping an image if size height or width is 120* 160
                    height, width , channel = image.shape
                    if height == 120 or width == 120:
                        image=image[20:140,:120,:]
                    #crop the images and resize them. Note that the images are of 2 different shape                     
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    #resizing all iamge to 100*100
                    image = cv2.resize(image,(100,100))
                    image = image/255
                    batch_data[folder,idx,:,:,0] = image[:,:,0]
                    #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = image[:,:,1]
                    #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = image[:,:,2]
                    #normalise and feed in the image
                
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do
        
        # write the code for the remaining data points which are left after full batches
        if len(folder_list) > num_batches * batch_size:
            x=len(img_idx)
            remaining_data_size = len(folder_list) - (num_batches * batch_size)
            batch_data = np.zeros((remaining_data_size,x,100,100,3))
            batch_labels = np.zeros((remaining_data_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(remaining_data_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (num_batches * batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (num_batches * batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                     
                    #cropping an image if size height or width is 120* 160
                    height, width , channel = image.shape
                    if height == 120 or width == 120:
                        image=image[20:140,:120,:]
                    #crop the images and resize them. Note that the images are of 2 different shape                     
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    #resizing all iamge to 100*100
                    image = cv2.resize(image,(100,100))
                    image = image/255
                    batch_data[folder,idx,:,:,0] = image[:,:,0]
                    #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = image[:,:,1]
                    #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = image[:,:,2]
                    #normalise and feed in the image
                
                batch_labels[folder, int(t[folder + (num_batches * batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do            


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [6]:
curr_dt_time = datetime.datetime.now()
train_path = 'Project_data/train'
val_path = 'Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 75 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 75


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [7]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
#Our import
from keras.regularizers import l2

#write your model here
cnn = Sequential()
cnn.add(Conv2D(32, (3, 3), padding='same', input_shape=(100,100,3)))
cnn.add(BatchNormalization())
cnn.add(Activation('relu'))
cnn.add(MaxPooling2D(pool_size=(2,2)))

cnn.add(Conv2D(64, (3, 3), padding='same', input_shape=(100,100,3)))
cnn.add(BatchNormalization())
cnn.add(Activation('relu'))
cnn.add(MaxPooling2D(pool_size=(2,2)))

cnn.add(Conv2D(128, (3, 3), padding='same', input_shape=(100,100,3)))
cnn.add(BatchNormalization())
cnn.add(Activation('relu'))
cnn.add(MaxPooling2D(pool_size=(2,2)))

cnn.add(Conv2D(128, (3, 3), padding='same', input_shape=(100,100,3)))
cnn.add(BatchNormalization())
cnn.add(Activation('relu')) 
cnn.add(MaxPooling2D(pool_size=(2,2)))
cnn.add(Dropout(0.25))

cnn.add(Conv2D(128, (3, 3), padding='same', input_shape=(100,100,3)))
cnn.add(BatchNormalization())
cnn.add(Activation('relu')) 
cnn.add(MaxPooling2D(pool_size=(2,2)))
cnn.add(Dropout(0.25))


#model.add(LSTM(100,return_sequences=True))
cnn.add(Flatten())
cnn.add(Dense(256, activation='relu'))
cnn.add(Dropout(0.5))
cnn.add(Dense(256, activation='relu'))
model = Sequential()
model.add(TimeDistributed(cnn, input_shape=(12, 100, 100,3)))
model.add(GRU(12))
model.add(Dropout(.2)) #added
model.add(Dense(5, activation='softmax'))

2022-02-23 15:53:36.305129: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-23 15:53:36.451981: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-23 15:53:36.452899: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-23 15:53:36.454309: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [8]:
optimiser = tf.keras.optimizers.Adam()
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 12, 256)           751296    
_________________________________________________________________
gru (GRU)                    (None, 12)                9720      
_________________________________________________________________
dropout_3 (Dropout)          (None, 12)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 65        
Total params: 761,081
Trainable params: 760,121
Non-trainable params: 960
_________________________________________________________________
None


Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [9]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [10]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr=0.0001) # write the REducelronplateau code here
#try different patience level
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [11]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [12]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/train ; batch size = 32


/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:17: DeprecationWarning:     `imread` is deprecated!
    `imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``imageio.imread`` instead.
2022-02-23 15:53:40.635229: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/75


2022-02-23 15:53:43.989923: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


20/21 [===========================>..] - ETA: 1s - loss: 1.5477 - categorical_accuracy: 0.3281

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:47: DeprecationWarning:     `imread` is deprecated!
    `imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``imageio.imread`` instead.


21/21 [==============================] - 40s 2s/step - loss: 1.5412 - categorical_accuracy: 0.3303 - val_loss: 2.4006 - val_categorical_accuracy: 0.2100

Epoch 00001: saving model to model_init_2022-02-2315_53_35.026342/model-00001-1.54125-0.33032-2.40055-0.21000.h5
Epoch 2/75
21/21 [==============================] - 31s 2s/step - loss: 1.3591 - categorical_accuracy: 0.3846 - val_loss: 2.1239 - val_categorical_accuracy: 0.2200

Epoch 00002: saving model to model_init_2022-02-2315_53_35.026342/model-00002-1.35911-0.38462-2.12394-0.22000.h5
Epoch 3/75
21/21 [==============================] - 31s 2s/step - loss: 1.2473 - categorical_accuracy: 0.4630 - val_loss: 2.4044 - val_categorical_accuracy: 0.2000

Epoch 00003: saving model to model_init_2022-02-2315_53_35.026342/model-00003-1.24733-0.46305-2.40442-0.20000.h5
Epoch 4/75
21/21 [==============================] - 29s 1s/step - loss: 1.1796 - categorical_accuracy: 0.5008 - val_loss: 2.2402 - val_categorical_accuracy: 0.2100

Epoch 00004:

In [ ]:
# MAX VALIDATION SET ACCURACY ACHIEVED AT
# Epoch 51/75
# 21/21 [==============================] - 30s 2s/step - loss: 0.0561 - categorical_accuracy: 0.9970 - val_loss: 0.3231 - val_categorical_accuracy: 0.9000
# Epoch 00051: saving model to model_init_2022-02-2315_53_35.026342/model-00051-0.05608-0.99698-0.32312-0.90000.h5